In [1]:
import requests
from urllib.request import Request, urlopen
import urllib
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import csv
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import os
import numpy as np
import re
from random import randint
from time import sleep


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import selenium


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/compat/_optional.py:106: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.0.0' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
PROJECT_ROOT = os.path.abspath(os.path.dirname('scrape.ipynb'))
DRIVER_BIN = os.path.join(PROJECT_ROOT, "chromedriver")
DRIVER_BIN

driver=selenium.webdriver.Chrome(executable_path=DRIVER_BIN)
driver.get('https://doordash.launchgiftcards.com/')


In [74]:
#make sure there is only one order
states = {'Alaska': 1,
'Alabama': 2,
'Arkansas': 3,
'American Samoa': 4,
'Arizona': 5,
'California': 6,
'Colorado': 7,
'Connecticut': 8,
'District of Columbia': 9,
'Delaware': 10,
'Florida': 11,
'Georgia': 12,
'Guam': 13,
'Hawaii': 14,
'Iowa': 15,
'Idaho': 16,
'Illinois': 17,
'Indiana': 18,
'Kansas': 19,
'Kentucky': 20,
'Louisiana': 21,
'Massachusetts': 22,
'Maryland': 23,
'Maine': 24,
'Michigan': 25,
'Minnesota': 26,
'Missouri': 27,
'Northern Mariana Islands': 28,
'Mississippi': 29,
'Montana': 30,
'North Carolina': 31,
'North Dakota': 32,
'Nebraska': 33,
'New Hampshire': 34,
'New Jersey': 35,
'New Mexico': 36,
'Nevada': 37,
'New York': 38,
'Ohio': 39,
'Oklahoma': 40,
'Oregon': 41,
'Pennsylvania': 42,
'Puerto Rico': 43,
'Rhode Island': 44,
'South Carolina': 45,
'South Dakota': 46,
'Tennessee': 47,
'Texas': 48,
'United States Minor Outlying Islands': 49,
'Utah': 50,
'Virginia': 51,
'Virgin Islands, U.S.': 52,
'Vermont': 53,
'Washington': 54,
'Wisconsin': 55,
'West Virginia': 56,
'Wyoming': 57}

In [226]:
purchase(sender_email='rlin20@uw.edu', sender_address='1123 19th Ave', city='Honolulu', state='New York', zipcode='96822', cardholder_name='Robi Lin', card_number='11235813', exp_date='05/20', cvv='112')

In [4]:
def purchase(sender_email, sender_address, city, state, zipcode, cardholder_name, card_number, exp_date, cvv):
    # Check to make sure only one order item
    if len(driver.find_elements_by_xpath("//div[@class='preview-order-item']")) == 1:
        # email
        driver.find_element_by_xpath("//input[@name='email']").send_keys(sender_email)
        # address
        driver.find_element_by_xpath("//input[@name='address']").send_keys(sender_address)
        # city
        driver.find_element_by_xpath("//input[@name='city']").send_keys(city)
        # state
        driver.find_element_by_xpath("//*[@id='pay-form']/form/div[2]/div/div[4]/div[1]/div/div/div/span[2]").click()
        driver.find_element_by_xpath("//input[@aria-activedescendant='react-select-2--option-0']").send_keys(state)
        driver.find_element_by_xpath("//input[@aria-activedescendant='react-select-2--option-0']").send_keys(Keys.ENTER)
        # zipcode
        driver.find_element_by_xpath("//input[@name='zip']").send_keys(zipcode)
        
        # cardholder_name
        zips = driver.find_element_by_xpath("//input[@name='zip']")
        zips.send_keys(Keys.TAB)
        cardHolder = driver.switch_to.active_element
        cardHolder.find_element_by_xpath("//input[@id='braintree__card-view-input__cardholder-name']").send_keys(cardholder_name)
        
        # card_number
        cardHolder.send_keys(Keys.TAB)
        cardNumber = driver.switch_to.active_element
        actions = ActionChains(driver)
        actions.send_keys(card_number)
        actions.perform()
        
        # exp_date
        cardNumber.send_keys(Keys.TAB)
        expDate = driver.switch_to.active_element
        actions = ActionChains(driver)
        actions.send_keys(exp_date)
        actions.perform()
        
        # cvv (security code of card)
        expDate.send_keys(Keys.TAB)
        securityCode = driver.switch_to.active_element
        actions = ActionChains(driver)
        actions.send_keys(cvv)
        actions.perform()
        
        # accept terms and conditions
        driver.find_element_by_xpath("//input[@name='terms']").click()
        while driver.find_element_by_xpath("//input[@name='terms']").get_attribute('value') == 'false':
            driver.find_element_by_xpath("//input[@name='terms']").click()
            sleep(randint(.02,1.5))
        
        # place order
        #driver.find_element_by_xpath("//button[@class='action-button action-button--purchase-now']").click()
    else:
        print('Error - more than one order-item')

In [3]:
def preFill(dollars, recipient_name, recipient_email, sender_name):
    amounts = {25: 1, 50: 2, 100: 3, 200: 4 }
    amount = amounts[dollars]
    gift_message = "Hi {}, please enjoy these credits! Courtesy of your donor, {}, brought to you by your friends at care37!".format(recipient_name, sender_name)
    driver.find_element_by_xpath("//span[@class='Select-arrow-zone']").click()
    driver.find_element_by_xpath("//*[@id='react-select-2--list']/div/div/div/div[{}]".format(amount)).click()
    driver.find_element_by_xpath("//*[@id='app']/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div[3]/div/div/form/div[1]/label[1]").click()
    driver.find_element_by_xpath("//input[@name='recipient_name']").send_keys(recipient_name)
    driver.find_element_by_xpath("//input[@name='recipient_email']").send_keys(recipient_email)
    driver.find_element_by_xpath("//*[@id='app']/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div[3]/div/div/form/div[4]/textarea").send_keys(gift_message)
    driver.find_element_by_xpath("//input[@name='sender_name']").send_keys(sender_name)
    driver.find_element_by_xpath("//button[@class='action-button']").click()

In [ ]:
preFill(25, 'robi', 'rlin20@uw.edu')